## Instruction

In [12]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


## 0. Import module

In [13]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

#import copy
#import math
import torch
import os
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image
#import numpy
#from numpy import linalg as LA
#import PIL
import seaborn as sns
sns.set(color_codes=True)

import sys
sys.path.append("../src/")
import model
from datasets import imagenet
from loss import FileterLoss
import config
#from utils.function import init_logging, init_environment, preprocess_image,\
#         recreate_image, get_lr, save_image
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map_for_comparision

In [30]:
# 0.Path
src_image_path = "./test_data/src.jpeg"
dst_image_path = "./test_data/dst.jpeg"
original_image_path = "./test_data/origin.jpeg"
resume = "037-0"
model_dir = "../saved/models/"
backbone = "vgg16"

In [40]:
# 1.Load Image & preprocess the imgs
original_image = np.array(Image.open(original_image_path).convert("RGB")).astype("float32")
src_image = np.array(Image.open(src_image_path).convert("RGB")).astype("float32")
dst_image = np.array(Image.open(dst_image_path).convert("RGB")).astype("float32")
imgs = np.array([original_image, src_image, dst_image])
imgs = imgs / 255.0
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
optimized_data = zscore(imgs, mean, std)

In [31]:
# Load model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = model.Network(backbone=backbone, num_classes=1000)
net.to(device)
# resume from model
resume_exp = resume.split("-")[0]
resume_epoch = resume.split("-")[1]
print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
ckpt = torch.load(resume_path, map_location=device)
net.load_state_dict(ckpt) 

> Use original fc
Resume from model from exp: 037 at epoch 0


<All keys matched successfully>

In [42]:
# Hyper-parameter
selected_layer = 3 
selected_filter = 15 
mode = "keep"
inter = True
rho = 0
regularization = "L1"
smoothing = "TotalVariation"
regular_ex = 1
alpha = 100.0
beta = 1.0
gamma = 1.0
delta = 0

In [43]:
# 2.Calculate Loss
criterion = FileterLoss(net, selected_layer, selected_filter, mode,
                        inter=inter, rho=rho,
                        regularization=regularization,
                        smoothing=smoothing, p=regular_ex, _print=print)

In [47]:
# optimized img-1
processed_inputs = torch.unsqueeze(torch.tensor(optimized_data[0]), 0)
original_inputs = torch.unsqueeze(torch.tensor(optimized_data[1]), 0)
selected_filter_loss, rest_fileter_loss, regularization_loss,  \
      smoothing_loss = criterion(processed_inputs, original_inputs)
loss = alpha * selected_filter_loss + beta * rest_fileter_loss + \
    gamma * regularization_loss + delta * smoothing_loss
print(selected_filter_loss)
print(rest_fileter_loss)
print(regularization_loss)
print(smoothing_loss)

Inter
tensor(204.1762, grad_fn=<MeanBackward0>)
tensor(2415.5542, grad_fn=<MeanBackward0>)
tensor(204787.3125)
tensor(22331.4805)


In [48]:
# optimized img-2
processed_inputs = torch.unsqueeze(torch.tensor(optimized_data[0]), 0)
original_inputs = torch.unsqueeze(torch.tensor(optimized_data[2]), 0)
selected_filter_loss, rest_fileter_loss, regularization_loss,  \
      smoothing_loss = criterion(processed_inputs, original_inputs)
loss = alpha * selected_filter_loss + beta * rest_fileter_loss + \
    gamma * regularization_loss + delta * smoothing_loss
print(selected_filter_loss)
print(rest_fileter_loss)
print(regularization_loss)
print(smoothing_loss)

Inter
tensor(208.7141, grad_fn=<MeanBackward0>)
tensor(6815.8491, grad_fn=<MeanBackward0>)
tensor(204787.3125)
tensor(22331.4805)


In [ ]:
# 3. Difference images